# Bloch-Torrey Equation

## Introduction

Here we solve the Bloch-Torrey equation on a unit square, with the diffusion coefficient $D(x)$, relaxation rate $R(x)$, and resonance frequency $\omega(x)$ all given as a generic functions.
The strong form of the Bloch-Torrey equation is given by

\begin{align}
    u_t &= \nabla \cdot (D \nabla u) - R u + \omega v  \quad x \in \Omega\\
    v_t &= \nabla \cdot (D \nabla v) - R v - \omega u  \quad x \in \Omega,
\end{align}

where $\vec{u}=[u,v]$ is the transverse magnetization, and $\Omega$ the domain.

We will consider homogeneous Neumann boundary conditions such that

\begin{align}
    \nabla u(x) \cdot n &= 0  \quad x \in \partial \Omega\\
    \nabla v(x) \cdot n &= 0  \quad x \in \partial \Omega,
\end{align}

where $\partial \Omega$ denotes the boundary of $\Omega$. The initial condition is given generically as

\begin{equation}
    \vec{u}(x,t=0) = \vec{u}_0 (x)  \quad x \in \Omega
\end{equation}


The resulting weak form is given by
\begin{align}
    \int_{\Omega} \vec{v} \cdot \vec{u}_t \, d\Omega
    &= -\int_{\Omega}
    -\vec{v} \cdot \nabla \cdot ( D \, \nabla \vec{u} ) +
    R \, \vec{v} \cdot \vec{u} -
    \omega \, \vec{v} \times \vec{u}
    \, d\Omega \\
    &= -\int_{\Omega}
    D \, \nabla \vec{v} : \nabla \vec{u} +
    R \, \vec{v} \cdot \vec{u} -
    \omega \, \vec{v} \times \vec{u}
    \, d\Omega + 
    \int_{\partial\Omega} \vec{v} \cdot (D\nabla\vec{u} \cdot \hat{n}) \, d\Gamma,
\end{align}
where $\vec{v}$ is a suitable test function.

In this notebook, we will assume homogeneous Neumann boundary conditions on all boundaries. Therefore, the final weak form is simply
\begin{align}
    \int_{\Omega} \vec{v} \cdot \vec{u}_t \, d\Omega
    = -\int_{\Omega}
    D \, \nabla \vec{v} : \nabla \vec{u} +
    R \, \vec{v} \cdot \vec{u} -
    \omega \, \vec{v} \times \vec{u}
    \, d\Omega
\end{align}

Note that, in two dimensions, the cross product is simply a scalar. However, `Tensors.jl` defines the two dimensional cross product by first extending the 2D vectors into 3D. Below, we use the symbol $\boxtimes$ to denote the scalar version, which is the same as taking the third component of the vector version

## Commented Program

Now we solve the problem in JuAFEM. What follows is a program spliced with comments.

First we load the JuAFEM package.

In [1]:
using Revise
using Traceur
using JuAFEM
using JuAFEM: vertices, faces, edges
using MATLAB
using DifferentialEquations
using BenchmarkTools

In [2]:
include("../Utils/geometry_utils.jl")
include("../Utils/mesh_utils.jl")
Revise.track("../Utils/geometry_utils.jl")
Revise.track("../Utils/mesh_utils.jl")

**Option #1**: Generate a simple grid with 20x20 triangular elements
using `generate_grid`. The generator defaults to the unit square,
so we don't need to specify the corners of the domain.

In [3]:
const n = 20
grid = generate_grid(Triangle, (n, n));

**Option #2**: External call to `MATLAB` to generate non-uniform grid with circles inside and convert to a `JuAFEM` grid.

In [4]:
bbox    = Rectangle((-1.0, -1.0), (1.0, 1.0));
centers = [(0.0, 0.0)];
rads    = [      0.5 ];

#bbox    = Rectangle((-1.0, -1.0), (1.0, 1.0));
#centers = [(-0.7, -0.9), (0.5, 0.5)];
#rads    = [        0.5,        0.4 ];

#bbox    = Rectangle((-1.0, -1.0), (1.0, 1.0));
#centers = [(0.0, -1.0), (0.0, 1.0)];
#rads    = [       0.5,        0.5 ];

circles = [Circle(c,r) for (c,r) in zip(centers,rads)]

Nmin = 250; # points for smallest circle
h0 = 2pi*minimum(rads)/Nmin; # approximate scale
eta = 5.0; # approx ratio between largest/smallest edges

In [5]:
mxcall(:addpath,0,"/home/coopar7/Documents/code/MatlabTools/FiniteElements/distmesh-jd")

In [6]:
revise()

In [7]:
fullgrid, subgrids = square_mesh_with_circles(bbox, circles, h0, eta, isunion=true);

In [8]:
#grid = subgrids[1]
grid = fullgrid

JuAFEM.Grid{2,3,Float64,3} with 22663 Triangle cells and 11450 nodes

### Diffusion coefficient $D(x)$, relaxation rate $R(x)$, and resonance frequency $\omega(x)$

In [9]:
const θ = π/2;
const cos²θ = cos(θ)^2;
const sin²θ = sin(θ)^2;

In [10]:
#@inline Dcoeff(x::Vec{2,T}) where T = ifelse(is_in_any_circle(x, circles), 3one(T), one(T))
@inline Dcoeff(x::Vec{2,T}) where T = one(T)
@inline Rdecay(x::Vec{2,T}) where T = ifelse(is_in_any_circle(x, circles), 3one(T), one(T))
@inline @fastmath @inbounds function omega(x::Vec{2,T}) where T
    ω = zero(T)
    for c in circles
        if is_in_circle(x,c)
            ω += T(3cos²θ-1)/6
        else
            r⁴ = (x⋅x)^2
            a² = radius(c)^2
            y²_x² = x[2]^2 - x[1]^2
            ω += T(0.5sin²θ) * a² * y²_x² / r⁴
        end
    end
    return ω
end

### Trial and test functions
A `CellValues` facilitates the process of evaluating values and gradients of
test and trial functions (among other things). Since the problem
is a scalar problem we will use a `CellScalarValues` object. To define
this we need to specify an interpolation space for the shape functions.
We use Lagrange functions (both for interpolating the function and the geometry)
based on the reference "cube". We also define a quadrature rule based on the
same reference cube. We combine the interpolation and the quadrature rule
to a `CellScalarValues` object.

In [11]:
const dim = 2
ip = Lagrange{dim, RefTetrahedron, 1}()
qr = QuadratureRule{dim, RefTetrahedron}(2)
qr_face = QuadratureRule{dim-1, RefTetrahedron}(2)
cellvalues = CellVectorValues(qr, ip);
facevalues = FaceVectorValues(qr_face, ip);

### Degrees of freedom
Next we need to define a `DofHandler`, which will take care of numbering
and distribution of degrees of freedom for our approximated fields.
We create the `DofHandler` and then add a single field called `u`.
Lastly we `close!` the `DofHandler`, it is now that the dofs are distributed
for all the elements.

In [12]:
dh = DofHandler(grid)
push!(dh, :u, 2)
close!(dh);

Now that we have distributed all our dofs we can create our tangent matrix,
using `create_sparsity_pattern`. This function returns a sparse matrix
with the correct elements stored.

In [13]:
K = create_sparsity_pattern(dh);
M = create_sparsity_pattern(dh);

We can inspect the pattern using the `spy` function from `UnicodePlots.jl`.
By default the stored values are set to $0$, so we first need to
fill the stored values, e.g. `K.nzval` with something meaningful.

In [14]:
#using UnicodePlots
#fill!(K.nzval, 1.0)
#spy(K; height = 25)

### Boundary conditions
In JuAFEM constraints like Dirichlet boundary conditions are handled by a `ConstraintHandler`. However, here we will have no need to directly enforce boundary conditions, since Neumann boundary conditions have already been applied in the derivation of the weak form.

### Assembling the linear system
Now we have all the pieces needed to assemble the linear system, $K u = f$.
We define a function, `doassemble` to do the assembly, which takes our `cellvalues`,
the sparse matrix and our DofHandler as input arguments. The function returns the
assembled stiffness matrix, and the force vector.

In [15]:
function doassemble(cellvalues::CellVectorValues{dim},
                    facevalues::FaceVectorValues{dim},
                    K::SparseMatrixCSC,
                    M::SparseMatrixCSC,
                    dh::DofHandler) where {dim}
    # We allocate the element stiffness matrix and element force vector
    # just once before looping over all the cells instead of allocating
    # them every time in the loop.
    n_basefuncs = getnbasefunctions(cellvalues)
    Ke = zeros(n_basefuncs, n_basefuncs)
    Me = zeros(n_basefuncs, n_basefuncs)
    
    # Next we create assemblers for the stiffness matrix `K` and the mass
    # matrix `M`. The assemblers are just thin wrappers around `K` and `M`
    # and some extra storage to make the assembling faster.
    assembler = start_assemble(K)
    assembler_M = start_assemble(M)
    
    # It is now time to loop over all the cells in our grid. We do this by iterating
    # over a `CellIterator`. The iterator caches some useful things for us, for example
    # the nodal coordinates for the cell, and the local degrees of freedom.
    @inbounds for cell in CellIterator(dh)
        # Always remember to reset the element stiffness matrix and
        # element mass matrix since we reuse them for all elements.
        fill!(Ke, 0)
        fill!(Me, 0)
        
        # Get the coordinates of the cell
        coords = getcoordinates(cell)
        
        # For each cell we also need to reinitialize the cached values in `cellvalues`.
        JuAFEM.reinit!(cellvalues, cell)
        
        # It is now time to loop over all the quadrature points in the cell and
        # assemble the contribution to `Ke` and `Me`. The integration weight
        # can be queried from `cellvalues` by `getdetJdV`, and the quadrature
        # coordinate can be queried from `cellvalues` by `spatial_coordinate`
        for q_point in 1:getnquadpoints(cellvalues)
            dΩ = getdetJdV(cellvalues, q_point)
            coords_qp = spatial_coordinate(cellvalues, q_point, coords)
            
            # calculate the heat conductivity and heat source at point `coords_qp`
            R = Rdecay(coords_qp)
            D = Dcoeff(coords_qp)
            ω = omega(coords_qp)
            
            # For each quadrature point we loop over all the (local) shape functions.
            # We need the value and gradient of the testfunction `v` and also the gradient
            # of the trial function `u`. We get all of these from `cellvalues`.
            for i in 1:n_basefuncs
                v  = shape_value(cellvalues, q_point, i)
                ∇v = shape_gradient(cellvalues, q_point, i)
                for j in 1:n_basefuncs
                    u = shape_value(cellvalues, q_point, j)
                    ∇u = shape_gradient(cellvalues, q_point, j)
                    vu = v ⋅ u
                    #@show u
                    #@show v
                    #@show typeof(u)
                    #@show typeof(v)
                    #@show ∇u
                    #@show ∇v
                    #@show typeof(∇u)
                    #@show typeof(∇v)
                    Ke[i, j] -= (D * ∇v ⊡ ∇u + R * vu - ω * v ⊠ u) * dΩ
                    Me[i, j] += vu * dΩ
                end
            end
        end
        
        # The last step in the element loop is to assemble `Ke` and `Me`
        # into the global `K` and `M` with `assemble!`.
        assemble!(assembler, celldofs(cell), Ke)
        assemble!(assembler_M, celldofs(cell), Me)
    end
    return K, M
end

doassemble (generic function with 1 method)

In [16]:
# # Now, loop over the edges of the cell for contributions to `Ke`
# #   If "Neumann Boundary" is a subset of boundary points, use:
# #     `onboundary(cell, face) && (cellid(cell), face) ∈ getfaceset(grid, "Neumann Boundary")`
# if evalfaceintegrals
#    for face in 1:nfaces(cell)
#        if !onboundary(cell, face)
#            # Initialize face values
#            reinit!(facevalues, cell, face)
# 
#            for q_point in 1:getnquadpoints(facevalues)
#                dΓ = getdetJdV(facevalues, q_point)
#                coords_qp = spatial_coordinate(facevalues, q_point, coords)
# 
#                # calculate the heat conductivity and heat source at point `coords_qp`
#                k_cond = conductivity(coords_qp)
#                kdΓ = k_cond * dΓ
# 
#                for i in 1:getnbasefunctions(facevalues)
#                    n = getnormal(facevalues, q_point)
#                    v = shape_value(facevalues, q_point, i)
#                    vkdΓ = v * kdΓ
#                    for j in 1:n_basefuncs
#                        ∇u = shape_gradient(facevalues, q_point, j)
#                        Ke[i,j] += (∇u⋅n) * vkdΓ
#                    end
#                end
#            end
#        end
#    end
# end

### Solution of the differential equation system
The last step is to solve the system. First we call `doassemble`
to obtain the global stiffness matrix `K` and force vector `f`.
Then, to account for the boundary conditions, we use the `apply!` function.
This modifies elements in `K` and `f` respectively, such that
we can get the correct solution vector `u` by using `\`.

In [17]:
fill!(K,0.0)
fill!(M,0.0)
K, M = doassemble(cellvalues, facevalues, K, M, dh);

In [18]:
Tf = eltype(K)
u0 = zeros(Tf, size(K,2))
u0[2:2:end] .= one(Tf)
tspan = (0.0,0.01);

In [19]:
du = similar(u0);
buffer = similar(u0);
Mchol = cholfact(M);
Mlu = lufact(M);

In [20]:
Ku!(du,u,p,t) = (K = p[1]; A_mul_B!(du, K, u))

Ku! (generic function with 1 method)

In [21]:
function Minv_Ku!(du,u,p,t)
    K, Mfact = p
    A_mul_B!(du, K, u)
    copy!(du, Mfact\du)
end

Minv_Ku! (generic function with 1 method)

In [23]:
p = (K, Mchol)
prob = ODEProblem(Minv_Ku!, u0, tspan, p)
@time sol = solve(prob, saveat=tspan)

retcode: Success
Interpolation: 1st order linear
t: 2-element Array{Float64,1}:
 0.0 
 0.01
u: 2-element Array{Array{Float64,1},1}:
 [0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0  …  0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0]                                                                                                                                
 [-0.00140341, 0.990048, -0.001399, 0.990048, -0.00137764, 0.990048, -0.00141658, 0.990048, -0.00144066, 0.990048  …  -0.000454903, 0.978697, -0.00176322, 0.989959, -0.00187025, 0.989918, -0.0016138, 0.975417, -0.00169982, 0.976687]

In [ ]:
p = (K,)
prob_Ku = ODEProblem(Ku!, u0, tspan, p, mass_matrix=M)
@time sol_Ku = solve(prob_Ku, ImplicitEuler(), saveat=tspan)

### Exporting to VTK
To visualize the result we export the grid and our field `u`
to a VTK-file, which can be viewed in e.g. [ParaView](https://www.paraview.org/).

In [ ]:
sizeof(u0)

In [ ]:
vtk_grid("bloch_torrey_equation", dh) do vtk
    vtk_point_data(vtk, dh, u)
end

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*